In [1]:
# Import packages
import numpy as np
import pandas as pd
from datetime import datetime
# Import my packages
import sys
import os
path_to_package = os.path.abspath(
    os.path.join('..')
)
sys.path.append(path_to_package)
from download_data import StockUniverse

In [36]:
# Test out __main__ with ['AAPL', 'MSFT']
test = StockUniverse()
test.download_data(['AAPL', 'MSFT'])
test.store_pickle(indicator_select=['O', 'V'], filename=f'../data/test_{datetime.today().strftime("%Y-%m-%d")}.pkl')
test.df

[*********************100%***********************]  2 of 2 completed
Saved file successfully at ../data/test_2020-10-23.pkl


Adj Close                   Close                    High  \
                  AAPL        MSFT        AAPL        MSFT        AAPL   
Date                                                                     
2017-01-03   27.548414   58.826855   29.037500   62.580002   29.082500   
2017-01-04   27.517582   58.563644   29.004999   62.299999   29.127501   
2017-01-05   27.657520   58.563644   29.152500   62.299999   29.215000   
2017-01-06   27.965857   59.071262   29.477501   62.840000   29.540001   
2017-01-09   28.222006   58.883255   29.747499   62.639999   29.857500   
...                ...         ...         ...         ...         ...   
2020-10-16  119.019997  219.660004  119.019997  219.660004  121.550003   
2020-10-19  115.980003  214.220001  115.980003  214.220001  120.419998   
2020-10-20  117.510002  214.649994  117.510002  214.649994  118.980003   
2020-10-21  116.870003  214.800003  116.870003  214.800003  118.709999   
2020-10-22  115.750000  214.889999  115.750000  214.889999  118.040001   

                               Low                    Open              \
                  MSFT        AAPL        MSFT        AAPL        MSFT   
Date                                                                     
2017-01-03   62.840000   28.690001   62.130001   28.950001   62.790001   
2017-01-04   62.750000   28.937500   62.119999   28.962500   62.480000   
2017-01-05   62.660000   28.952499   62.029999   28.980000   62.189999   
2017-01-06   63.150002   29.117500   62.040001   29.195000   62.299999   
2017-01-09   63.080002   29.485001   62.540001   29.487499   62.759998   
...                ...         ...         ...         ...         ...   
2020-10-16  222.289993  118.809998  219.320007  121.279999  220.149994   
2020-10-19  222.300003  115.660004  213.720001  119.959999  220.419998   
2020-10-20  217.369995  115.629997  213.089996  116.199997  215.800003   
2020-10-21  216.919998  116.449997  213.119995  116.669998  213.119995   
2020-10-22  216.059998  114.589996  211.699997  117.449997  213.929993   

               Volume            
                 AAPL      MSFT  
Date                             
2017-01-03  115127600  20694100  
2017-01-04   84472400  21340000  
2017-01-05   88774400  24876000  
2017-01-06  127007600  19922900  
2017-01-09  134247600  20382700  
...               ...       ...  
2020-10-16  115393800  26057900  
2020-10-19  120639300  27625800  
2020-10-20  124423700  22753500  
2020-10-21   89946000  22724900  
2020-10-22  101709700  22334100  

[959 rows x 12 columns]

In [37]:
# Test read_pickle()
test = StockUniverse()
test.read_pickle(filename=f'../data/test_{datetime.today().strftime("%Y-%m-%d")}.pkl')

Open                 Volume          
                  AAPL        MSFT       AAPL      MSFT
Date                                                   
2017-01-03   28.950001   62.790001  115127600  20694100
2017-01-04   28.962500   62.480000   84472400  21340000
2017-01-05   28.980000   62.189999   88774400  24876000
2017-01-06   29.195000   62.299999  127007600  19922900
2017-01-09   29.487499   62.759998  134247600  20382700
...                ...         ...        ...       ...
2020-10-16  121.279999  220.149994  115393800  26057900
2020-10-19  119.959999  220.419998  120639300  27625800
2020-10-20  116.199997  215.800003  124423700  22753500
2020-10-21  116.669998  213.119995   89946000  22724900
2020-10-22  117.449997  213.929993  101709700  22334100

[959 rows x 4 columns]

In [62]:
# Test calc_normalized_returns(['O'])
_ = test.calc_normalized_returns(['O', 'V'])
new_col_names = [each_col_name + ' Returns'  for each_col_name in _.columns.get_level_values(0).unique()]
_.columns.

FrozenList([['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], ['AAPL', 'MSFT']])